# MongoDB

In [1]:
# MongoDB
import pymongo
from pymongo import MongoClient
import pandas as pd
import json

# Pretty Printing - GH: EdwardBetts/pprint_color.py
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import Terminal256Formatter
from pprint import pformat

def cprint(obj):
    print(highlight(pformat(obj), PythonLexer(), Terminal256Formatter()))

print('Mongo version '+ pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.ryan
collection = db.restaurants

Mongo version 3.7.1


In [2]:
cursor = collection.find().sort('type_of_food',pymongo.DESCENDING).limit(3)
for doc in cursor:
    cprint(doc)
    print()

{'URL': 'http://www.just-eat.co.uk/restaurants-anhdao/menu',
 '_id': ObjectId('55f14312c7447c3da7051efe'),
 'address': '106-108 Kingsland Road',
 'address line 2': 'London',
 'name': 'Anh Dao',
 'outcode': 'E2',
 'postcode': '8DP',
 'rating': 4,
 'type_of_food': 'Vietnamese'}


{'URL': 'http://www.just-eat.co.uk/restaurants-anhdao/menu',
 '_id': ObjectId('55f14312c7447c3da7051eff'),
 'address': '106-108 Kingsland Road',
 'address line 2': 'London',
 'name': 'Anh Dao',
 'outcode': 'E2',
 'postcode': '8DP',
 'rating': 4,
 'type_of_food': 'Vietnamese'}


{'URL': 'http://www.just-eat.co.uk/restaurants-amrit-sweetcentre-le5/menu',
 '_id': ObjectId('55f14312c7447c3da7051eb3'),
 'address': '236 Green Lane Road',
 'address line 2': 'Leicestershire',
 'name': 'Amrit Sweet Centre',
 'outcode': 'LE5',
 'postcode': '4PA',
 'rating': 4,
 'type_of_food': 'Vegetarian'}




Neato.

# Aggregation

In [3]:
# Up for breakfast?

pipeline = [
    {
        "$group":{
            "_id":"$type_of_food",
            "Average Rating":{"$avg":"$rating"},
            "Count":{"$sum":1}
        }
    },
    {
        "$sort":{
            # "Average Rating":-1,
            "Count": -1
        }
    }
]

agg = collection.aggregate(pipeline)
ratings = pd.DataFrame(list(agg))
ratings = ratings.set_index("_id")
ratings.head(10)

,Average Rating,Count
_id,,
Curry,5.036158,902
Pizza,4.914141,500
Chinese,4.893678,174
Kebab,4.885621,154
Fish & Chips,5.036697,116
American,4.617021,95
Turkish,4.918919,74
Lebanese,4.805970,70
Chicken,4.410000,53


In [4]:
ratings.tail()

,Average Rating,Count
_id,,
Pasta,6.0,1
Cakes,5.5,1
Nigerian,4.5,1
Punjabi,6.0,1
Spanish,4.5,1


# Well, that's pretty neat. 

Another one:

In [5]:
pipeline = [
    {"$match": {"type_of_food":"Breakfast"}}
]

agg = collection.aggregate(pipeline)
breakfast = pd.DataFrame(list(agg))
breakfast

,URL,_id,address,address line 2,name,outcode,postcode,rating,type_of_food
0,http://www.just-eat.co.uk/restaurants-133-take...,55f14312c7447c3da7051b36,133 Fullarton Street,Irvine,133 Takeaway,KA12,8DG,Not yet rated,Breakfast
1,http://www.just-eat.co.uk/restaurants-alfiesca...,55f14312c7447c3da7051d66,161 Helen Street,Glasgow,Alfies Cafe - Collection Only,G51,3HD,Not yet rated,Breakfast
2,http://www.just-eat.co.uk/restaurants-alldaybr...,55f14312c7447c3da7051dc8,227 Lower House Lane,Liverpool,All Day Breakfast Bar,L11,2SF,4.5,Breakfast
3,http://www.just-eat.co.uk/restaurants-annescaf...,55f14312c7447c3da7051f27,64 Anne Road,Smethwick,Anne's Cafe,B66,2NY,5.5,Breakfast
4,http://www.just-eat.co.uk/restaurants-benvenut...,55f14313c7447c3da70523b7,Bell Parade,West Wickham,Benvenuti - Collection Only,BR4,0RH,Not yet rated,Breakfast
5,http://www.just-eat.co.uk/restaurants-big-dadd...,55f14313c7447c3da7052475,2 Norris Street,Cheshire,Big Daddy House,WA2,7RL,Not yet rated,Breakfast
6,http://www.just-eat.co.uk/restaurants-bigphill...,55f14313c7447c3da7052499,Old Club Building,Tyne and Wear,Big Phil'lers Deli,NE27,0EP,5,Breakfast
